In [1]:
import pandas as pd

user_data = pd.read_csv("all_user_data_c_50_95.csv")

In [2]:
user_data.drop(user_data.columns[[0]], axis=1, inplace=True)
user_data.head()

,id,name,editcount,status
0,15,!!,1,1
1,228,!dea4u,1,1
2,387,"""Country"" Bushrod Washington",1652,0
3,582,"""Yahia""",1023,0
4,829,$1LENCE D00600D,18642,0


In [3]:
user_data = user_data[user_data['status']>0]
user_data = user_data[:1500]  # same as df.head(10)

In [4]:
user_data.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,id,editcount,status
count,1500,1500,1500
mean,1.85977e+06,1,1
std,959506,0,0
min,15,1,1
25%,976432,1,1
50%,1.8929e+06,1,1
75%,2.66532e+06,1,1
max,3.31915e+06,1,1


In [7]:
path = 'user_data_50_95/rev_list_' + '15' + '-d.json'
#print(path)
try:
    with open(path, 'r') as infile:
        data = json.loads(infile.read())

        if len(data) > 0:
            for d in data:
                del d['next_rev']

            graph_for = "longevityRev"

            series = pd.DataFrame(data=data)
            series = series[['pageid', 'timestamp', graph_for]]
            series = series[series.longevityRev >= 0]
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
                print(series)
except Exception as e:
        print('error :')

       pageid             timestamp  longevityRev
6       31306  2001-12-13T21:39:32Z           2.0
8       21473  2001-12-13T22:20:14Z           0.0
9       31306  2001-12-13T22:47:03Z          12.0
11      10948  2001-12-14T17:13:00Z           7.0
12      10948  2001-12-14T18:25:08Z           1.0
13      13606  2001-12-14T19:50:43Z           0.0
14      21272  2001-12-14T19:59:49Z          48.0
15      10948  2001-12-14T20:05:58Z           3.0
16       7463  2001-12-14T22:00:15Z           0.0
17      19640  2001-12-14T22:07:28Z          48.0
18      10948  2001-12-17T15:30:42Z           2.0
19       6310  2002-01-09T22:27:03Z          48.0
20      21785  2002-01-10T22:18:28Z           7.0
21      21785  2002-01-11T15:43:50Z           2.0
22      21785  2002-01-11T19:22:43Z          40.0
23      33794  2002-01-14T21:10:23Z          48.0
24      21785  2002-01-15T15:51:25Z          48.0
25      22107  2002-01-15T16:10:53Z           2.0
26      26541  2002-01-15T20:41:13Z          17.0


In [5]:

import os.path
import numpy as np
from trust_score import TrustScore
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import json

y_pred = []
y_test=[]
cou = 0

for row in user_data.iterrows():
    path = 'user_data_50_95/rev_list_' + str(row[1]['id']) + '-d.json'
    #print(path)
    try:
        with open(path, 'r') as infile:
            data = json.loads(infile.read())

            if len(data) > 0:
                for d in data:
                    del d['next_rev']

                graph_for = "longevityRev"

                series = pd.DataFrame(data=data)
                series = series[['pageid', 'timestamp', graph_for]]
                series = series[series.longevityRev >= 0]
                if len(series) > 9:
                    print('User ID: ', str(row[1]['id']))
                    print(series.head())
                    series['Trust'] = TrustScore([series[graph_for],48]).calculate()
                    series['longevityRevN'] = series['longevityRev'].shift(-1)
                    series['longevityRevP'] = series['longevityRev'].shift(1)
                    series = series[series.longevityRevP >= 0]
                    series = series[series.longevityRevN >= 0]
                    series = series[series.longevityRev >= 0]

                    seriesTrain = series[0:-1]
                    seriesTest = series[len(series)-1:]
                    #print(series)
                    regressor = LinearRegression()
                    regressor.fit(seriesTrain[['Trust', 'longevityRev','longevityRevP']], seriesTrain['longevityRevN'])

                    # To retrieve the intercept:
                    #print(regressor.intercept_)
                    # For retrieving the slope:
                    #print(regressor.coef_)

                    pred = regressor.predict(seriesTest[['Trust', 'longevityRev','longevityRevP']])
                    print('Data Count: ', len(seriesTrain))
                    print('Test Data: ')
                    print(seriesTest[['Trust', 'longevityRev','longevityRevP']])
                    print('predicted: ',pred, ' actual: ', seriesTest['longevityRevN'].values[0])
                    print("##########################")
                    if(pred>=0):
                        y_test.append(seriesTest['longevityRevN'].values[0])
                        y_pred.append(pred)
    except Exception as e:
        #print('error :',e.message)
        cou= cou+1

print(str(cou))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('R2 Score:', metrics.r2_score(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

User ID:  15
    pageid             timestamp  longevityRev
6    31306  2001-12-13T21:39:32Z           2.0
8    21473  2001-12-13T22:20:14Z           0.0
9    31306  2001-12-13T22:47:03Z          12.0
11   10948  2001-12-14T17:13:00Z           7.0
12   10948  2001-12-14T18:25:08Z           1.0
Data Count:  291
Test Data: 
        Trust  longevityRev  longevityRevP
348  0.114996          13.0            2.0
predicted:  [24.08948511]  actual:  26.0
##########################
User ID:  228
    pageid             timestamp  longevityRev
0    19137  2002-02-20T10:31:24Z           8.0
1    99800  2007-04-22T05:01:26Z          48.0
2     1519  2008-08-25T06:50:15Z           6.0
3  3385969  2009-01-19T05:37:41Z           0.0
4  2228434  2009-01-21T15:11:47Z          48.0
Data Count:  7
Test Data: 
      Trust  longevityRev  longevityRevP
8  0.733753          48.0            1.0
predicted:  [34.46816514]  actual:  48.0
##########################
User ID:  1400
   pageid             timestamp  l